In [2]:
import re

In [3]:
import pickle
import pandas as pd

### Compound names from MeSh substance headings


In [4]:
mesh_compounds = ['dihydropyridines?', 'benzothiazepines?', 'phenyl[- ]?alkyl[- ]?amines?', 'AE0047' , 'Amlodipine' , 'Amrinone' , 'anandamide' , 'anipamil' , 'azimilide' , 'Bencyclane' , 'benidipine' , 'Bepridil' , 'berbamine' , 'canadine' , 'carboxyamido-triazoles?' , 'caroverine' , 'CD 349' , 'cilnidipine' , 'Cinnarizine' , 'clentiazem' , 'clevidipine' , 'Conotoxins' , 'Conus magus toxin' , 'darodipine' , 'dauricine' , 'devapamil' , 'Diltiazem' , 'dotarizine' , 'efonidipine' , 'emopamil' , 'enpiperate' , 'eperisone' , 'falipamil' , 'fantofarone' , 'fasudil' , 'Felodipine' , 'fenamic acid' , 'Fendiline' , 'Flunarizine' , 'fosfedil' , 'FTX' , 'gabapentin' , 'Gallopamil' , 'IH 764-3' , 'Isradipine' , 'KT 362' , 'lacidipine' , 'lamotrigine' , 'lercanidipine' , 'Lidoflazine' , 'lomerizine' , 'manidipine' , 'manoalide' , 'mepirodipine' , 'Mibefradil' , 'monatepil' , 'N-(2-guanidinoethyl)-5-isoquinolinesulfonamide' , 'naftopidil' , 'Nicardipine' , 'Nifedipine' , 'niguldipine' , 'niludipin' , 'nilvadipine' , 'Nimodipine' , 'Nisoldipine' , 'Nitrendipine' , 'norverapamil' , 'ochratoxin A' , 'octylonium' , 'Agatoxins?' , 'osthol' , 'oxodipine' , 'Perhexiline' , 'pinaverium' , 'piperidines?' , 'PN 202-791' , 'pranidipine' , 'pregabalin' , 'Prenylamine' , 'R 56865' , 'risedronic acid' , 'ryodipine' , 'SAN 202791' , 'sesamodil' , 'stepholidine' , 'temiverine' , 'terodiline' , 'tetrahydropalmatine' , 'tetrandrine' , 'Tiapamil' , 'tolfenamic acid' , 'tranilast' , 'Verapamil' , 'Win 55212-2' , 'ziconotide']
mesh_compounds.remove('N-(2-guanidinoethyl)-5-isoquinolinesulfonamide')

In [5]:
len(mesh_compounds)

94

In [6]:
mesh_compounds.append('N-\(2-guanidinoethyl\)-5-isoquinolinesulfonamide')

In [7]:
mesh_compounds = [i.lower() for i in mesh_compounds]

### Preparing Wisniowska compounds to add to the list above


In [8]:
Wisniowska_suppl = pd.read_csv('Wisniowska_suppl.txt', sep = '\t')

In [9]:
master_list = [i.lower() for i in Wisniowska_suppl['Compound'].unique()[2:]]

In [10]:
len(master_list)

69

In [11]:
master_list.extend(mesh_compounds)

In [12]:
len(master_list)

164

In [13]:
master_list = list(set(master_list))

In [14]:
len(master_list)

148

In [15]:
master_list.remove('sr33557 (fantofarone)')

In [16]:
master_list.append('sr33557')

In [17]:
master_list.append('fantofarone')

In [18]:
master_list.remove('vanoredxine (gbr-12909)')

In [19]:
master_list.append('vanoredxine')

In [20]:
master_list.append('gbr-12909')

In [21]:
len(master_list)

150

### Define highlighting dictionaries

In [26]:
broader_terms = [
    {
          'name': 'chem'
        , 'terms': ['analog\w*', 'ligands?', 'binding', 'potency', 'potencies', 'affinit(?:y|ies)', 'displacement', 'displacing' ]
        , 'color': 'LightSeaGreen'
    }
    , {
          'name':'IC50s'
         ,'terms': [r'[IEXA][CD](?:50|₅₀)s?', r'[IEXA][CD]\(50\)s?', r'inhibitory activit(?:y|ies)', r'\bKi\b']
         ,'color': 'Goldenrod'}
    , {
          'name': 'calcium'
        , 'terms': ['calcium' 
                    

                    , 'Ca2?(\+)+[- ]channels?'
                    , '(?:calcium|ca)[ -](channels?|currents?)'
                    , 'Ca2?(\+)+'
                    , 'Ca2?(\+)+[- ](channels?|currents?)'
                    , 'Ca²⁺'
                    , 'L-type'
                    , 'LTCCs?'
                    ,'ICa, ?L'
                    , 'I\(Ca\)'
                    , 'I\(Ca,L\)'
                    , 'I\(Ca\(L\)\)'
                    ,'\(ICa\)'
                    ,'I\(Ca1.2\)'
                    , r'\bICa\b'
                    , 'DHPRs?'
                    , 'Cav[0-9]\.[0-9a-z]+'
                    , 'DHP receptors?'
                    , 'DHP'
                    , 'Ca\(2\+\)'
                    , r'\bCa(?:\b|-)'
                    
                    
                    , 'voltage-dependent (?:ca|calcium)'
                    , 'voltage-gated (?:ca|calcium)'
                    , 'Ca\(2\+\)[- ]?channel[- s]'
                    , 'Ca\(2\+\)[ -]channels?'
                    ,'calcium[- ]entry'
                    , 'Ca2?(\+)+'
                   ]
        , 'color': 'blue'
    }
    , {
          'name': 'other_terms'
        , 'terms': ['(?:modulator|antagonis|inhibitor|block)\w*', 'currents?', 'channels?']
        , 'color': 'RoyalBlue'
    }
    ,{
        'name':'interesting_tissue'
        , 'terms': ['cardiac', 'hearts?', 'cardio[\w-]+', 'atri(?:al|um)', 'ventricles?', 'ventricular', 'myocardi(?:al|um)', 'papillary']
        , 'color': 'Magenta'
    }
    , {
          'name':'unwanted_tissue'
        , 'terms':['ileal','ileum', 'smooth muscle', 'vascular muscle',  'nervous (?:tissues?|systems?)', r'\bnerves?\b', 'CNS', 'brain', 'contractions?', 'neuron\w*', 'skeletal muscle', 'vasodilatory', r'\bvascular\b', 'vaso-?(?:dilat|constrict|relax)\w+', 'constriction', 'dilator\w*', 'bronchiol\w*', 'aorta']
        , 'color':'red'
    } 
    , {
        'name':'effect'
        , 'terms': ['inotrop\w+', 'lusitrop\w+', 'dromotrop\w+', 'tachycard\w+', 'bradycard\w+', '(?:anti[- ]?)?arrhythmi\w+', 'tdp', 'torsade de pointes', 'QT', 'phasic', 'tonic', 'voltage[- ](?:sensitive|dependent)', 'use[- ]dependent']
        , 'color': 'RosyBrown'
    }
    , {
        'name':'other_channels'
        , 'terms': ['sodium', r'\bNa\b', 'potassium', r'\bK\+', r'\bK\b', 'N-type', 'T-type', '(?:[PQ]|P/Q)-type', 'R-type','ATPase' ]
        , 'color': 'Crimson'
    }
    , {
        'name':'assays'
        , 'terms': ['(?:patch|voltage)[- ]clamp\w*', 'IonWorks', 'electrophysiolog\w*', 'patchxpress']
        , 'color': 'Chocolate'
    }
    , {
        'name':'species'
        , 'terms': ['guinea[- ]pig', 'rabbit', r'\brat\b', 'mouse', 'dog', 'human', 'pig']
        , 'color': 'MediumOrchid'
    },
    {
        'name': 'chemicals'
        ,'terms': master_list
        ,'color':'MediumSeaGreen'
    }
]

In [27]:
with open('terms_for_highlighting.pkl', 'wb') as f:
    
    pickle.dump(broader_terms, f)     

In [24]:
p = re.compile('IC₅₀|IC50|IC\(50\)', re.I)

In [25]:
p.search('IC₅₀')

<_sre.SRE_Match object; span=(0, 4), match='IC₅₀'>